In [ ]:
# ##################################################
# The first step is to create a topic coverage matrix
# over the time slices
# used to run a granger test on topics
# ##################################################

# create a topic coverage matrix preset to 0
topiccoverage = []
i = 0
while i < len(docs_per_timeslice) :
    y = 0
    thistopic = []
    while y < num_topics:
        thistopic.append(float(0.0))
        y += 1
    topiccoverage.append(thistopic.copy())
    i += 1
    
# get the topic coverage per timeslice per doc from the model
timeslice = 0
for timeslicedocs in docs_per_timeslice :
    # for each doc in this timeslice
    for doc in timeslicedocs :
        # get the probability matrix
        probs = model.get_document_topics(bow[doc]).copy()
        #its a sparse array, prob[0] is the topic and prob[1] is the probabiltiy
        for prob in probs :
            topiccoverage[timeslice][prob[0]] += prob[1]
    timeslice += 1

# ##################################################
# Run the granger test on the topic coverage created above
#
# INPUT:
# timeslicetokencounts - we have the word coverage for each model token
# topiccoverage - and now we have the topic coverage
# topics - the current topic word probabilities
#
# OUTPUT:
# collected statistics, and update topic probabilities
# updated array newtopics = []
#
# run the iteration
# ##################################################   
newtopics = []
purity = float(0.0)
purity_count = 0
#lowtopics = []
sigtopics = []

causality_confidence = float(0.0)
causality_count = 0
sig_causality_confidence = float(0.0)
sig_causality_count = 0
#savelowpvalue = float(2.0) $$$
for ii in range(0, num_topics) :
    #
    # run the granger test on the topic coverage
    # save significant topics
    #
    tempgc = grangercausalitytests([[bets[i],topiccoverage[i][ii]] for i in range(0, len(bets))], the_lag, verbose=False)

    #
    # check the lag - for now we just grab the smallest p-value in the lag
    #
    lowpvalue = float(2.0)
    lowlag = 0
    for yy in range(1,the_lag+1) :
        stats = tempgc.get(yy)[0].get('ssr_ftest')
        if (stats[1] < lowpvalue) :
            lowlag = yy
            lowpvalue = stats[1]
    # grab the causality_confidence for every topic
    causality_confidence += lowpvalue
    causality_count += 1
    #lowtopics.append(ii)
    if lowpvalue < low_threshold : # lob_threshold is one of the system params, default 95% or .05 p-value - pretty standard
        sigtopics.append(ii)
        # grab the causality_conficence for sig topics
        sig_causality_confidence += lowpvalue
        sig_causality_count += 1
        #if savelowpvalue > lowpvalue : $$$
            #savelowpvalue = lowpvalue $$$
#
# if this is the largest causality_confidence
# save the topics for the output of the engine
#
if causality_confidence/causality_count < mostsigconf :
    mostsigconf = causality_confidence/causality_count
    mostsigtopics = sigtopics.copy()
    mostsigtopicwords = []
    for ii in range (0, len(sigtopics)) :
        mostsigtopicwords.append(model.get_topic_terms(sigtopics[ii], 10).copy())
    #print ("Significant Topics: ", len(sigtopics))
    file_name = runname + str(iteration) 
    vis_name = save_path + file_name + ".sav"
    print ("\nSignificant Topics: ", sigtopics, "\n")

#
# ok, now we have a list of significant topics
# we are going to adjust the probabilities 
#
for topic in sigtopics :
    #print(sigtopics)
    wordprobstats = []
    tot = np.longdouble(0)
    count = 0

    # we adjust the probability based upon the prerun granger tests on the word streams
    for wordprob in topics[topic] :
        #pgranger is the pre-run word stream granger tests
        sig = float(wordprob * pgranger[count])
        tot += wordprob #keep the total for normalization
        
        # save the word list for normalization
        wordlist = [abs(sig), pgranger[count], count, wordprob]
        wordprobstats.append(wordlist.copy())
        count += 1
    for wordprobstat in wordprobstats :
        wordprobstat[0] = wordprobstat[0]/tot # normalize

    #
    # sort the probabilitys
    #
    wordprobstats = sorted(wordprobstats, key = lambda x: float(x[0]), reverse = True)

    pos_count = 0
    neg_count = 0
    neg_percent = float(0.0)
    pos_percent = float(0.0)
    neg_wordprob = []
    pos_wordprob = []
    hit_limit = 0
    # create new topics, start with zero prob
    for wordprobstat in wordprobstats :
        if hit_limit == 0 : 
            if wordprobstat[1] > 0  :
                pos_wordprob.append(wordprobstat.copy())
                pos_percent += wordprobstat[0]
                pos_count += 1
            if wordprobstat[1] < 0   :
                neg_wordprob.append(wordprobstat.copy())
                neg_percent += wordprobstat[0]
                neg_count += 1
            if (neg_percent + pos_percent) > drop_percent or wordprobstat[0] < .0001:
                hit_limit = 1
    #print("OY Pos count: ", pos_count)    
    #print("OY Neg count: ", neg_count)    
    addpurity = 0
    if pos_count > neg_count * ignore_little_counts :
        # create new topics, start with zero prob
        createtopic = zeroprobs.copy()

        for wordprob in pos_wordprob :
            createtopic[wordprob[2]] = wordprob[0]/pos_percent
            #createtopic[wordprob[2]] = 1.0
        newtopics.append(createtopic.copy())
        addpurity = 1
        
    if neg_count > pos_count * ignore_little_counts :
        # create new topics, start with zero prob
        createtopic = zeroprobs.copy()

        for wordprob in neg_wordprob :
            createtopic[wordprob[2]] = wordprob[0]/neg_percent
            #createtopic[wordprob[2]] = 1.0
        newtopics.append(createtopic.copy())
        addpurity = 1
        
     
    ppurity = 0
    npurity = 0
    if pos_count + neg_count != 0 :
        ppurity = pos_count / (pos_count + neg_count)
        npurity = neg_count / (pos_count + neg_count)
    if (ppurity != 0) :
        ppurity = (ppurity * math.log(ppurity, 10))
    if (npurity != 0) :
        npurity = (npurity * math.log(npurity, 10))
    entropy =  npurity + ppurity
    #print(float(100 + (100 * entropy)))
    topic_purity = float(100 + (100 * entropy))
    purity += topic_purity
    purity_count += addpurity
    #print("Topic purity = ", topic_purity)
     

#print (newtopics)
causality_confidence = 1-causality_confidence/causality_count 
print("causality_confidence = ", causality_confidence)
run_confidence.append(causality_confidence.copy())
sig_causality_confidence = 1-sig_causality_confidence/sig_causality_count 
print("sig_causality_confidence = ", sig_causality_confidence)
#run_confidence.append(causality_confidence.copy())


purity = purity/purity_count
print("purity = ", purity)
run_purity.append(purity)

#%run ITMTF_iterate.ipynb

# ##################################################
#
# add buffers - num_buffers using bufferprob
#
# ##################################################
# add the buffers
z = 0
while z < num_buffers:
    newtopics.append(bufferprob.copy())
    z += 1
num_topics = len(newtopics)

print ("New Topics: ", len(newtopics))

# ##################################################
#
# using the new topics with updated probabilies
# return to the model
#
# ##################################################
